# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 8: Decision Trees

Implement a trainer for a single decision tree and an ensemble of decision tress, called Random Forest. Much of the code has already been completed in previous assignments. Once completed, you should have some learners to use for the Final Project.

The trainer implementation will happen in the following stages:

1. Create a ```NumericSplitter``` and a ```CategoricalSplitter``` which create a ```NumericSplit``` and a ```CategoricalSplit```, respecively. 
2. Create a ```DecisionTreeLearner``` which implements a recursive trainer given a list of possible splits
3. Add a ```predict``` method to the decision tree class that predicts for new examples
4. Using your ```DecisionTreeLearner``` as a weak learner, implement Bagging and a random subset of features to train a ```RandomForestTrainer```.
5. Implement the ```AUCMetric``` which helps you draw an ROC curve.
6. Apply $k$-fold Cross Validation to find the best parameters for two datasets. 

In [1]:
require './assignment_lib'

def load_german_credit_dataset; JSON.parse(File.read('german-credit.json')); end
def load_iris_dataset(); read_sparse_data_from_csv "iris"; end

puts "**Iris Dataset**"
puts "Features", load_iris_dataset()["features"], ""
puts "First example", load_iris_dataset()["data"].first

puts "\n\n**German Credit Dataset**"
puts "Features", load_german_credit_dataset()["features"], ""
puts "First example", load_german_credit_dataset()["data"].first

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

**Iris Dataset**
Features
["sepal_length", "sepal_width", "petal_length", "petal_width"]

First example
{"features"=>{"sepal_length"=>7.0, "sepal_width"=>3.2, "petal_length"=>4.7, "petal_width"=>1.4}, "label"=>1}


**German Credit Dataset**
Features
["checking_account", "loan_duration", "credit_history", "purpose", "credit_amount", "savings", "job_tenure", "installment_to_salary", "personal_status_gender", "other_debtors", "residence_tenure", "property", "age", "other_installments", "housing", "existing credits", "job", "dependents", "has_telephone", "is_foreign_worker"]

First example
{"id"=>0, "label"=>1, "features"=>{"checking_account"=>"(-inf,0)", "loan_duration"=>6.0, "credit_history"=>"critical account", "purpose"=>"radio/television", "credit_amount"=>1169.0, "savings"=>"none", "job_tenure"=>"[7,inf)", "installment_to_salary"=>4.0, "personal_status_gender"=>"m_single", "other_debtors"=>"none", "residence_tenure"=>4.0, "property"=>"real_estate", "age"=>67.0, "other_installments"=>

# Question 1

Copy the various pieces you have already implemented for decision trees

## Question 1.1 (1 Point)

Copy **your** implementation of ```class_distribution```, mean, and stdev from previous assignments

In [2]:
def class_distribution dataset
  #puts dataset
  classes = Hash.new {|h,k| h[k] = 0}
  dataset.each do |item|
    classes[item["label"]]=1+classes[item["label"]]
  end
  
  result={}
  classes.each do |key,array|
    result[key]=array.to_f/dataset.size.to_f
  end
  
  return result
end

def mean x
  sum=0
  x.each do |item|
    sum+=item.to_f
  end
  
  return sum.to_f/(x.size)
end

def stdev x
  sum=0
  mean1=mean(x)
  x.each do |item|
    sum+=(item-mean1)**2
  end

  return (sum.to_f/(x.size-1))**0.5
end


:stdev

In [3]:
### TESTS ###
def test_d5732b
  iris = load_iris_dataset()
  t1_iris_dist = class_distribution iris["data"]
  t1_iris_num_classes = 3
  n = t1_iris_num_classes.times do |cls|
    assert_in_delta t1_iris_dist[cls], 0.33333, 1e-4
  end
  assert_equal t1_iris_num_classes, n
end
test_d5732b()

## Question 1.2 (1 Point)

Copy **your** implementation of ```entropy``` from previous assignments

In [4]:
def entropy dist
  ent=0
  dist.each do |key,array|
    if array==0
      return 0.0
    end
    ent+=-array*Math.log(array)
  end
  
  ent
end

:entropy

In [5]:
### TESTS ###
def test_4b0b32()
  iris = load_iris_dataset()
  t2_iris_dist = class_distribution iris["data"]
  t2_iris_entropy = entropy t2_iris_dist
  assert_in_delta(1.0986122886681096, t2_iris_entropy, 1e-4)
  

  german_credit = load_german_credit_dataset()
  german_credit_dist = class_distribution german_credit["data"]
  german_credit_entropy = entropy german_credit_dist
  assert_in_delta(0.6108643020548935, german_credit_entropy, 1e-4)
end

test_4b0b32()

## Question 1.3 (1 Point)

Copy **your** implementation of ```information_gain``` from previous assignments

In [6]:
def information_gain h0, splits
 size = Hash.new {|h,k| h[k] = 0}
  sum = Hash.new {|h,k| h[k] = 0}
  total=0
  
  splits.each do |key, array|
    total+=array.size
  end
  
  splits.each do |key, array|
    sum[key]+=entropy(class_distribution(array))
    size[key]=array.size
  end
  
  result=0
  
  size.each do |key,array|
    size[key]=size[key].to_f/total
  end
  
  splits.each do |key, array|
    result+=sum[key]*size[key]
  end

  return h0-result
end

:information_gain

In [7]:
def test_21236c()
  iris = load_iris_dataset()
  t3_random_split = iris["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
  t3_entropy = entropy(class_distribution(iris["data"]))

  assert_true(!t3_entropy.nil?)
end
test_21236c()

# Question 2 Splits

Implement a ```NumericSplit``` and a ```CategoricalSplit``` based on methods you implemented in previous assignments. Although much of the code will be reused, we will need to rearrange it.

The decision tree uses the ```Splitter``` to find and create the best ```Split``` for a feature. The ```Split``` holds the information needed to define the split and check whether an example shown go down one path or another. 


## Question 2.1 (5 Points)

Implement the ```CategoricalSplit```, which can split a dataset by the values of a categorical value. 

For example, consider splitting on the feature "f1". 

For the input as follows:
```ruby
[ 
    {"features" => {"f1" => "a"}}, 
    {"features" => {"f1" => "b"}},
    {"features" => {"f2" => "c"}}
]
```

The splits are stored in a hash where the key is a name of the path and the values are all examples under that path.
```ruby
{ 
    "f1 == 'a'" => [{"features" => {"f1" => "a"}}],
    "f1 == 'b'" => [{"features" => {"f1" => "b"}}]
}
```

Notes:

1. Skip any example which has a missing value for the features
1. Use the pattern ```@path_pattern % [@feature_name, feature_value]``` to assign a string-valued name to the split.

In [8]:
class CategoricalSplit
  attr_reader :feature_name
  
  def initialize fname
    @feature_name = fname
    @path_pattern = "%s == '%s'"
  end
  
  def to_s
    "Categorical[#{@feature_name}]"
  end

  def split_on_feature examples
    
    splits = Hash.new {|h,k| h[k] = []}
    examples.each do |item|
      item["features"].each do |key2,array2|
        if key2==@feature_name
          @path_pattern = "%s == '%s'" % [key2, array2.to_s]
          splits[@path_pattern]=(splits[@path_pattern]).append(item)
        end
      end
    end

    return splits
  end
end

:split_on_feature

In [9]:
def test_af2e0c()  
  f1_split = CategoricalSplit.new "f1"

  example1 = {"features" => {"f1" => "a"}}
  example2 = {"features" => {"f1" => "b"}}
  example3_missing = {"features" => {"f2" => "c"}}
  examples = [example1, example2, example3_missing]
  
  splits = f1_split.split_on_feature examples
  puts "Splits", splits.keys  
  split_sizes = splits.values.collect {|v| v.size}
  puts split_sizes
  
  assert_equal 2, splits.size, "Returns all possible values for f1 feature"
  assert_equal 1, splits["f1 == 'a'"].size, "a"
  assert_equal 1, splits["f1 == 'b'"].size, "b"
end
test_af2e0c()

Splits
["f1 == 'a'", "f1 == 'b'"]
[1, 1]


In [10]:
def test_b08a8b()  
  job_split = CategoricalSplit.new "job"
  
  german_credit = load_german_credit_dataset()  
  examples = german_credit["data"]
  
  splits = job_split.split_on_feature examples
  puts "Splits for german_credit.jobs", splits.keys
  
  split_sizes = splits.values.collect {|v| v.size}
  puts split_sizes
  
  assert_equal 4, splits.size, "Returns all possible values for job feature"
  assert_equal 22, splits["job == 'unemployed'"].size, "unemployed"
  assert_equal 200, splits["job == 'unskilled'"].size, "unskilled"
end
test_b08a8b()

Splits for german_credit.jobs
["job == 'skilled'", "job == 'unskilled'", "job == 'management'", "job == 'unemployed'"]
[630, 200, 148, 22]


## Question 2.2 (4 Points)

Add a ```test``` method to your ```CategoricalSplit``` which considers an example and returns the string-valued name of the path to follow. If the example does not have a value for this feature, return ```nil```.

For example, for an example with feature "f1", defined as follows:

```ruby
{"features" => {"f1" => "a"}}
```

The ```test``` method returns the name of the appropriate path to follow based on the feature. Note that the string name is available through the ```@path_pattern``` member variable.

Output:
```ruby
"f1 == 'a'"
```

Notes:

1. Note that the categorical value could be generated dynamically. 

In [11]:
class CategoricalSplit
  def test example 

    if example["features"][@feature_name]
      return "%s == '%s'" % [@feature_name, example["features"][@feature_name]]
    end 
      
    return nil

  end
end

:test

In [12]:
def test_746297()  
  f1_split = CategoricalSplit.new "f1"

  example1 = {"features" => {"f1" => "a"}}
  example2 = {"features" => {"f1" => "b"}}  
  example3 = {"features" => {"f2" => "c"}}
  
  assert_equal "f1 == 'a'", f1_split.test(example1), "Example 1"
  assert_equal "f1 == 'b'", f1_split.test(example2), "Example 2"  
  assert_equal nil, f1_split.test(example3), "Example 3"
end
test_746297()

## Question 2.3 (5 Points)

Implement ```NumericSplit``` which splits a dataset based on a provided numeric value. The ```split_on_feature``` method is very similar to the ```split_on_numeric_value``` from previous assignments. You may refactor **your** implementation of ```split_on_numeric_value```.

The splits are stored in a hash where the key is a name of the path and the values are all examples under that path. For example, consider the iris dataset, with a feature name "petal_length" and the value1 1.7. The ```split_on_feature``` method should return:

```ruby
{
    "petal_length >= 1.7" => [...],
    "petal_length < 1.7" => [...],
}
```

Notes:
1. For numeric values, any feature which is missing should be assumed to have a value of zero.
1. The path names are stored in a member array for numeric features because they are deterministic.

In [13]:
class NumericSplit
  attr_reader :feature_name, :split_point, :paths
  def initialize fname, value
    @feature_name = fname
    @split_point = value
    @split_point_str = "%.2g" % @split_point
    @paths = ["#{@feature_name} < #{@split_point_str}", "#{@feature_name} >= #{@split_point_str}"]
  end
  
  def to_s
    "Numeric[#{@feature_name} <=> #{@split_point_str}]"
  end

  def split_on_feature examples
    splits = Hash.new 

    splits={@paths[0]=>[], @paths[1]=>[]}

    examples.each do |item|
      if item["features"][@feature_name]
        y=item["features"][@feature_name]
      else
        y=0.0
      end
      if y<@split_point
        splits[@paths[0]] << item
      end
      if y>=@split_point
        splits[@paths[1]] << item
      end
    end
  
    return splits
  end
end

:split_on_feature

In [14]:
def test_c02859()  
  t21_split = NumericSplit.new "petal_length", 1.7
  
  iris = load_iris_dataset()
  t21_iris_splits = t21_split.split_on_feature iris["data"]
  puts "Splits", t21_iris_splits.keys  
  split_sizes = t21_iris_splits.values.collect {|v| v.size}
  puts split_sizes

  
  t21_split_sizes = t21_iris_splits.values.collect {|v| v.size}.sort
  t21_num_splits = 2
  assert_equal t21_num_splits, t21_split_sizes.size
  assert_equal 44, t21_split_sizes[0]
  assert_equal 106, t21_split_sizes[1]
end
test_c02859()

Splits
["petal_length < 1.7", "petal_length >= 1.7"]
[44, 106]


In [15]:
def test_17d874()
  iris = load_iris_dataset()
  split = NumericSplit.new "petal_length", 1.7
  t5_iris_splits = split.split_on_feature iris["data"]
  t5_split_sizes = t5_iris_splits.values.collect {|v| v.size}.sort

  # Checks the information gain for this split
  t5_iris_entropy = entropy(class_distribution(iris["data"]))
  t5_iris_information_gain = information_gain t5_iris_entropy, t5_iris_splits
  assert_in_delta 0.48280104455013506, t5_iris_information_gain, 5e-2
end
test_17d874()

## Question 2.3 (5 Points)

Add the ```test``` method to the ```NumericSplit``` which will return a string-valued path, corresponding to one of the values of the ```@paths``` array defined earlier. 

For example, consider the input as follows:

```ruby
{"features" => {"petal_length" => 0.7}}
```

The ```test``` method should return:

```ruby
"petal_length < 1.7" 
```

Notes:

1. If the feature value does not exist for the example, return ```nil```.

In [16]:
class NumericSplit
  def test example
      
    if example["features"][@feature_name]
      y=example["features"][@feature_name]
      if y==nil
        return nil
      end
      if y<@split_point
        return @paths[0]
      elsif y>=@split_point
        return @paths[1]
      end
    end
  
  end
end

:test

In [17]:
def test_64e0d8()
  splitter = NumericSplit.new "petal_length", 1.7
  t22_x_left = {"features" => {"petal_length" => 0.7}}
  assert_equal splitter.paths[0], splitter.test(t22_x_left), "< 1.7"

  t22_x_right = {"features" => {"petal_length" => 1.71}}
  assert_equal splitter.paths[1], splitter.test(t22_x_right), ">= 1.7"

  t22_x_right_eq = {"features" => {"petal_length" => 1.7}}
  assert_equal splitter.paths[1], splitter.test(t22_x_right_eq), "= 1.7"
end
test_64e0d8()

In [18]:
def test_cf6969()
  splitter = NumericSplit.new "petal_length", 1.7
  x_missing = {"features" => {"petal_width" => 0.7}}
  assert_equal nil, splitter.test(x_missing), "Handle missing value"
end
test_cf6969()

# Question 3

Implement the ```CategoricalSplitter``` and ```NumericSplitter```. A ```Splitter``` checks that a ```Split``` can be applied, creates the ```Split```, and calculates the information gain. Note that the ```matches?``` function is provided for you and determines whether a split can apply. 

## Question 3.1 (5 points)

Implement the ```CategoricalSplit``` class, whose ```create_split``` method which takes the parent node's entropy, some examples and creates a ```CategoricalSplit``` on that feature name. It also calculates the information gain on the split.

For example, given a dataset and a feature name of "job", this should return the following:

```ruby
{"split" => CategoricalSplit(...), "information_gain" => 0.1234}
```

Notes:
1. You should call your ```information_gain``` function here.

In [19]:
class CategoricalSplitter
  def matches? examples, feature_name
    has_feature = examples.select {|r| r["features"].has_key? feature_name} 
    return false if has_feature.empty?    
    return has_feature.all? do |r| 
      r["features"].fetch(feature_name, 0.0).is_a?(String)
    end
  end
  
  def create_split examples, parent_entropy, feature_name
    
    if(matches?(examples, feature_name))
      categorical_split = CategoricalSplit.new feature_name
      splits = categorical_split.split_on_feature examples
      ig = information_gain parent_entropy, splits
      return {"split" => categorical_split, "information_gain" => ig}
    end
     
  end
  
  
end

:create_split

In [20]:
def test_f251f0()
  german_credit = load_german_credit_dataset()
  examples = german_credit["data"]
  feature_name = "job"
  
  h0 = entropy(class_distribution(examples))
  splitter = CategoricalSplitter.new 
  split_result = splitter.create_split examples, h0, feature_name

  puts "Split Result:", split_result
  
  split = split_result["split"]
  info_gain = split_result["information_gain"]

  assert_not_nil split
  assert_in_delta 0.0009269851357646131, info_gain, 1e-2
end 
test_f251f0()

Split Result:
{"split"=>#<CategoricalSplit:0x00000000039e7890 @feature_name="job", @path_pattern="job == 'skilled'">, "information_gain"=>0.0009269851357646131}


## Question 3.2 (5 Points

Implement the ```NumericSplitter``` class. Given a feature, create_split finds the best feature value and creates the split. This behavior was implemented in a previous assignment, so adapt **your** ```find_split_point_numeric``` method to implement ```create_split```. 


```ruby
{"split" => NumericSplit(...), "information_gain" => 0.1234}
```

Notes:
1. In previous versions of the ```find_split_point_numeric```, we set missing values to zero. We will do the same thing here. However in the ```matches?``` function, we require at least one non-missing value. 
1. Return a ```nil``` if there is no available split point.

In [21]:
class NumericSplitter
  def matches? examples, feature_name
    has_feature = examples.select {|r| r["features"].has_key? feature_name} 
    return false if has_feature.empty?    
    return has_feature.all? do |r| 
      r["features"].fetch(feature_name, 0.0).is_a?(Numeric)
    end
  end
  
  def create_split examples, parent_entropy, feature_name  
    
    if(matches?(examples, feature_name))

      sorted_values = examples.collect {|r| r["features"][feature_name]}.uniq.sort
      ig_max=-1
      t_max=-1

      threshold = []
      iG = []
      sorted_values.each do |item|
        threshold << item
        split1 = NumericSplit.new(feature_name, item)
        iG << information_gain(parent_entropy, split1.split_on_feature(examples))
        if iG.last>ig_max
          ig_max=iG.last
          t_max=threshold.last
        end
      end

      split = NumericSplit.new feature_name, t_max
      ig=information_gain(parent_entropy, split.split_on_feature(examples))

      return {"split" => split, "information_gain" => ig}
    
    end
    
  end
end

:create_split

In [22]:
def test_f35008()
  iris = load_iris_dataset()
  t6_iris_entropy = entropy(class_distribution(iris["data"]))
  splitter = NumericSplitter.new 
  split_result = splitter.create_split iris["data"], t6_iris_entropy, "sepal_width"
  
  split = split_result["split"]
  info_gain = split_result["information_gain"]

  assert_in_delta 3.4, split.split_point, 1e-2, "Iris best split"

end
test_f35008()

In [23]:
def test_207f35()
  gc = load_german_credit_dataset()
  gc_entropy = entropy(class_distribution(gc["data"]))  
  splitter = NumericSplitter.new 
  split_result = splitter.create_split gc["data"], gc_entropy, "age"

  split = split_result["split"]
  info_gain = split_result["information_gain"]
  
  assert_not_nil split
  assert_in_delta 0.007817315714005901, info_gain, 1e-3
  assert_in_delta 26.0, split.split_point, 1e-2
end 
test_207f35()

In [24]:
def test_eedbae()
  parent_entropy = 1.0
  splitter = NumericSplitter.new 
  empty_examples = []
  split_result = splitter.create_split empty_examples, parent_entropy, "none"

  assert_nil split_result, "Empty examples"
  
  one_example = [{"features" => {"f1" => 1.0}, "label" => 1}]
  split_result = splitter.create_split one_example, parent_entropy, "one"

  assert_nil split_result, "One example"
end 
test_eedbae()

## Question 4

The core of the decision tree is the ```DecisionNode```, which is used in training and evaluation.

During training, the ```DecisionNode``` temporarily holds an array of examples, finds the best split and creates the splits. 

During evaluation, the node applies the test to find which of its children should be consulted for the leaf node. A leaf node is a node without any children.

## Question 4.1 (5 Points)

Implement the ```score``` method. When in a leaf node, the score for a particular class is calculated as the class distribution for the training data in that node. Because a decision tree is a multi-class classifier, we need to specify the class label when calling the ```score``` function. You should assume that there is no smoothing so if label isn't in the class distribution, the score is zero.

The score is the posterior class distribution given that an example is in the leaf $l$, defined as follows:

# $P(c \mid l) = \frac{ \left| \left\{x \mid c(x) = c \wedge x \in l \right\} \right| }{ \left| \left\{x\in l \right\} \right| }$

where $c(x)$ is the class label for example $x$.

Why is the score function for a leaf returning a constant and does not need to even look at the example?
  
**A**. Decision tree is overfitting by memorizing the training data and thus will not generalize

**B**. The code is wrong and it should be looking at the example

**C**. Decision tree assumes that all examples in the leaf have the same class distribution

**D**. It does look at the example, but it stores a class member variable so it does not need to be provided as an argument


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answers is "Z", write the following:_

```ruby
def answer_ce174b()
    %w(Z)
end
```

In [25]:
def answer_673f8b()
  %w(C)
end

:answer_673f8b

In [26]:
assert_not_nil answer_673f8b()

In [27]:
class DecisionNode
  attr_reader :children, :examples, :split, :node_entropy, :node_class_distribution
  
  def initialize examples
    @examples = examples
    @node_class_distribution = class_distribution examples    
    @node_entropy = entropy (@node_class_distribution)
    @children = Hash.new
  end
  
  def is_leaf?
    self.children.empty?
  end
  
  def set_split thisSplit
    @split=thisSplit
  end
      
  def score positive_class_label
    
    scores=Hash.new {|h,k| h[k] = 0}

    if @node_class_distribution[positive_class_label]==nil
      return 0
    else return @node_class_distribution[positive_class_label]
    end
  end

end

:score

In [28]:
def test_42e1b1()
  iris = load_iris_dataset()

  # Check the first split for iris
  examples = iris["data"]
  root = DecisionNode.new examples

  assert_true root.is_leaf?, "Single-node root is a leaf"
  
  assert_in_delta 0.3333, root.score(0), 1e-3, "Score for class 0"
  assert_in_delta 0.3333, root.score(1), 1e-3, "Score for class 1"
  assert_in_delta 0.3333, root.score(2), 1e-3, "Score for class 2"
  assert_in_delta 0.0, root.score(3), 1e-3, "Score for class 3 -- there is no class 3"  
end
test_42e1b1()

## Question 4.2 (10 Points)

Add ```all_possible_splits``` to the ```DecisionNode```. This considers an array of feature names and an array of splitters (```CategoricalSplitter``` or ```NumericSplitter```). Next, apply all splitters, if they match, to each feature. Then, apply a set of filters:
1. Remove any split which is ```nil```
1. Remove any split where the information gain is not strictly $> 0$. 

Return all possible splits that meet the criteria. 

For example, given the iris dataset and the ```NumericSplitter```, return the following best splits for each feature:

```ruby
    [
        {"split"=>NumericSplit(feature_name: "petal_width", split_point: X), "information_gain" => 0.636514}, 
        {"split"=>NumericSplit(feature_name: "petal_length", split_point: X),  "information_gain"=>0.636514},
        {"split"=>NumericSplit(feature_name: "sepal_length", split_point: X), "information_gain"=>0.38624},
        ...
    ]
```

Note that the result above would be the same if we used passed both the ```CategoricalSplitter``` and ```NumericSplitter``` because only the ```NumericSplitter``` will ```matches?``` features in the iris dataset.

In [29]:
class DecisionNode
  def all_possible_splits feature_names, splitters
    all_splits = []
    
    splitters.each do |split|
      feature_names.each do |item|

        split_result = split.create_split @examples, @node_entropy, item
        if !split_result or (split_result["split"]==nil and split_result["information_gain"]==nil) 
          split_result=nil
        else 
          all_splits+=[split_result]
        end
      end
    end

    all_splits.delete(nil)
    
    return all_splits
  end
end

:all_possible_splits

In [30]:
def test_e3c5cb()
  iris = load_iris_dataset()

  # Check the first split for iris
  feature_names = iris["features"]
  examples = iris["data"]
  root = DecisionNode.new examples
  
  splitters = [NumericSplitter.new]
  all_splits = root.all_possible_splits feature_names, splitters

  assert_equal 4, all_splits.size, "Calculate all possible splits"  
  sorted_splits = all_splits.sort_by {|split| split["information_gain"]}.reverse
  
  puts sorted_splits
  best_split = sorted_splits.first
  
  puts "Best split", best_split
  split = best_split["split"]
  info_gain = best_split["information_gain"]

  assert_not_nil split
  assert_in_delta 0.6365141682948128, info_gain, 1e-2, "Returns top info gain"
  assert_false %w(petal_length petal_width).index(split.feature_name).nil?, "Find one of the top features"
end
test_e3c5cb()

[{"split"=>#<NumericSplit:0x0000000003691740 @feature_name="petal_width", @split_point=1.0, @split_point_str="1", @paths=["petal_width < 1", "petal_width >= 1"]>, "information_gain"=>0.6365141682948128}, {"split"=>#<NumericSplit:0x000000000394fe28 @feature_name="petal_length", @split_point=3.0, @split_point_str="3", @paths=["petal_length < 3", "petal_length >= 3"]>, "information_gain"=>0.6365141682948128}, {"split"=>#<NumericSplit:0x0000000003c41848 @feature_name="sepal_length", @split_point=5.6, @split_point_str="5.6", @paths=["sepal_length < 5.6", "sepal_length >= 5.6"]>, "information_gain"=>0.3862442664692113}, {"split"=>#<NumericSplit:0x0000000003b71738 @feature_name="sepal_width", @split_point=3.4, @split_point_str="3.4", @paths=["sepal_width < 3.4", "sepal_width >= 3.4"]>, "information_gain"=>0.18570201019349364}]
Best split
{"split"=>#<NumericSplit:0x0000000003691740 @feature_name="petal_width", @split_point=1.0, @split_point_str="1", @paths=["petal_width < 1", "petal_width >= 1

In [31]:
def test_377c12()
  german_credit = load_german_credit_dataset()

  # Check the first split for iris
  feature_names = german_credit["features"]
  examples = german_credit["data"]
  root = DecisionNode.new examples
  
  splitters = [CategoricalSplitter.new]
  all_splits = root.all_possible_splits feature_names, splitters

  assert_equal 13, all_splits.size, "Calculate all possible splits"  
  sorted_splits = all_splits.sort_by {|split| split["information_gain"]}.reverse
  
  puts sorted_splits
  best_split = sorted_splits.first
  
  puts "Best split", best_split
  split = best_split["split"]
  info_gain = best_split["information_gain"]

  assert_not_nil split
  assert_in_delta 0.06566796091172744, info_gain, 1e-2, "Returns top info gain"
  assert_false %w(checking_account).index(split.feature_name).nil?, "Find one of the top features"
end
test_377c12()

[{"split"=>#<CategoricalSplit:0x00000000039c6e88 @feature_name="checking_account", @path_pattern="checking_account == '[0,200)'">, "information_gain"=>0.06566796091172744}, {"split"=>#<CategoricalSplit:0x000000000393b8d8 @feature_name="credit_history", @path_pattern="credit_history == 'critical account'">, "information_gain"=>0.030233554614261138}, {"split"=>#<CategoricalSplit:0x0000000002f89790 @feature_name="savings", @path_pattern="savings == '[100,500)'">, "information_gain"=>0.01948760776931857}, {"split"=>#<CategoricalSplit:0x00000000036cd4c0 @feature_name="purpose", @path_pattern="purpose == 'car_used'">, "information_gain"=>0.017254887066567637}, {"split"=>#<CategoricalSplit:0x00000000039071c8 @feature_name="property", @path_pattern="property == 'other'">, "information_gain"=>0.011773233742666589}, {"split"=>#<CategoricalSplit:0x0000000003c066a8 @feature_name="job_tenure", @path_pattern="job_tenure == 'unemployed'">, "information_gain"=>0.009081837924813763}, {"split"=>#<Catego

## Question 4.3 (5 Points)

Given a specific split, possibly created from ```all_possible_splits``` above, implement ```split_node!```. Ruby's convention of adding a "!" to a method indicates that the method changes the state of the object. In this case, the ```split_node!``` method calls the ```split_on_feature``` method of the split and creates children ```DecisionNode```s. The ```@children``` member variable is a hash which maps each path calculated by the split to ```DecisionNode```. 

Given input examples as follows:
```ruby
[
    {"features" => {"f1" => "a", "f2" => "d"}},
    {"features" => {"f1" => "a", "f2" => "c"}},
    {"features" => {"f1" => "b", "f2" => "c"}}
]
```

Children should be as follows:
```ruby
{
    "f1 == 'a'" => DecisionNode(...),
    "f1 == 'b'" => DecisionNode(...)
}
```

Note:
1. The ```@examples``` data held by the node is freed after splitting to reduce memory consumption.

In [32]:
class DecisionNode
  def split_node! split    
    @split = split
    
    mySplit=@split.split_on_feature(@examples)

    mySplit.each do |key,array|
      @children[key]=DecisionNode.new(array)
    end

    #@examples = nil
  end
end

:split_node!

In [33]:
def test_cfc360()  
  # Check the first split for iris
  example1 = {"features" => {"f1" => "a", "f2" => "d"}}
  example2 = {"features" => {"f1" => "a", "f2" => "c"}}
  example3 = {"features" => {"f1" => "b", "f2" => "c"}}
  
  examples = [example1, example2, example3]
  
  root = DecisionNode.new examples  
  split = CategoricalSplit.new "f1"  
  root.split_node! split

  children = root.children
  puts children
  assert_equal 2, children.size, "Creates 2 children"  
  assert_equal 2, children["f1 == 'a'"].examples.size, "Child a"
  assert_equal 1, children["f1 == 'b'"].examples.size, "Child b"
end
test_cfc360()

{"f1 == 'a'"=>#<DecisionNode:0x00000000036d5918 @examples=[{"features"=>{"f1"=>"a", "f2"=>"d"}}, {"features"=>{"f1"=>"a", "f2"=>"c"}}], @node_class_distribution={nil=>1.0}, @node_entropy=0.0, @children={}>, "f1 == 'b'"=>#<DecisionNode:0x00000000036d56e8 @examples=[{"features"=>{"f1"=>"b", "f2"=>"c"}}], @node_class_distribution={nil=>1.0}, @node_entropy=0.0, @children={}>}


# Question 5

With the ```DecisionNode``` complete, we are ready to build the decision tree trainer. The decision tree is an example of a ```Learner``` which we are defining below. The ```Learner``` interface will enable us to compare several different models and combinations of models. For this and all future assignments, we will implement new algorithms as a ```Learner```. The interface is defined below. Note that Ruby does not have an explicit interface, so we use a mixin module.

In [34]:
module Learner  
  attr_reader :parameters
  def train train_dataset    
  end
  def predict example
  end
  def evaluate eval_dataset
  end
end
  

:evaluate

The contract for a ```Learner``` is defined as follows:

* ```parameters```: A hash containing parameter values used for training. Keep only model hyper-parameters in this hash. 
* ```train```: Trains the model and keeps any model state in the object. Post training, the learner can be used as a trained model. The same learner can be re-trained if needed.
* ```predict```: Predicts on a single example, expects the model to be trained first. Assumes the model is a binary classifier and returns a single score related to the positive class. 
* ```evaluate```: Evaluates the model on a full dataset and returns an array of (score, label) pairs.


In ```evaluate```, the output is as follows:

```ruby
[
    [0.12345, 0],
    [0.4562, 1]
]
````

where the first entry in each element the score (output of predict) and the second is the label. Use 1 for positive and 0 for negative regardless of how the class is defined in the dataset.

## Question 5.1 (10 points)

The ```train``` method for a decison tree is simple.  It calls a recursive method ```grow_tree``` which starts at a ```parent``` and proceeds to grow the parent by considering all possible splits, take the best, and create children. Then, it tries to grow each child on that child's subset of the data. 

As a recursive function, ```grow_tree``` has 3 termination criteria:

1. The tree is too deep--no more remaining depth
2. No more splits available
3. No more than ```min_size``` examples in the parent.

A summary method method has been provided for you to print the resulting tree. For a ```max_depth``` of 1, this should produce a single leaf node. On the iris dataset, this is as follows:

```json
{
  "leaf": true,
  "class_distribution": {
    "1": 0.3333333333333333,
    "2": 0.3333333333333333,
    "0": 0.3333333333333333
  }
}
```

For a ```max_depth``` of 2, this should produce a tree with exactly one split. On the iris dataset, this is as follows:

```json
{
  "leaf": false,
  "split": "Numeric[petal_length <=> 3]",
  "children": {
    "petal_length < 3": {
      "leaf": true,
      "class_distribution": {
        "0": 1.0
      }
    },
    "petal_length >= 3": {
      "leaf": true,
      "class_distribution": {
        "1": 0.5,
        "2": 0.5
      }
    }
  }
}
```

Notes:
1. We are implementing a fixed height (depth) tree. This is necessary both to prevent overflow but we will also use it for testing. Keep in mind that depth is **not** a typical hyperparameter. We are just using it to simplify the calculation. In practice, set the depth to a really large number and use the ```min_size``` as the true hyperparameter.

In [35]:
class DecisionTreeLearner
  include DecisionTreeHelper
  include Learner  
  attr_reader :root
  
  def initialize positive_class_label, min_size: 10, max_depth: 50
    @splitters = [CategoricalSplitter.new, NumericSplitter.new]
    @parameters = {"min_size" => min_size, "max_depth" => max_depth}
    @positive_class_label = positive_class_label
  end
    
  def train dataset
    @feature_names = dataset["features"]
    examples = dataset["data"]
    @root = DecisionNode.new examples
    grow_tree @root, @parameters["max_depth"]
  end

  def grow_tree parent, remaining_depth

    if parent.examples.size<@parameters["min_size"] or !@feature_names.size or remaining_depth<1
      return
    end
   
    all_splits=parent.all_possible_splits(@feature_names,@splitters)
    sorted_splits=all_splits.sort_by{|split| split["information_gain"]}.reverse
    
    best_split=sorted_splits.first
    split=best_split["split"]
    
    decreased_depth=remaining_depth-1
    parent.set_split(split)
    
    if (decreased_depth>0)
      parent.split_node! (split)
      children=parent.children
      children.each do |child|
        decreased_depth=remaining_depth-1
        child_decision_node=child[1]
        grow_tree(child_decision_node,decreased_depth)
      end
    end
  end
    
end

:grow_tree

In [36]:
def test_594f6e()
  iris = load_iris_dataset()
  
  model = DecisionTreeLearner.new 1, min_size: 10, max_depth: 1
  model.train iris
  tree = model.root

  puts "Root Only Tree", model.to_s

  assert_true tree.children.empty?
  assert_true tree.is_leaf?
end

test_594f6e()

Root Only Tree
{
  "leaf": true,
  "class_distribution": {
    "1": 0.3333333333333333,
    "2": 0.3333333333333333,
    "0": 0.3333333333333333
  }
}


In [37]:
def test_1c728f()
  iris = load_iris_dataset()
  model = DecisionTreeLearner.new 1, min_size: 10, max_depth: 2
  model.train iris
  root = model.root

  puts "Two-level Tree", model.to_s

  assert_false root.is_leaf?
  assert_not_nil root.split
  assert_equal 2, root.children.size
  assert_true(root.children.values.all? {|leaf| leaf.is_leaf?})
end
test_1c728f()

Two-level Tree
{
  "leaf": false,
  "split": "Numeric[petal_width <=> 1]",
  "children": {
    "petal_width < 1": {
      "leaf": true,
      "class_distribution": {
        "0": 1.0
      }
    },
    "petal_width >= 1": {
      "leaf": true,
      "class_distribution": {
        "1": 0.5,
        "2": 0.5
      }
    }
  }
}


In [38]:
def test_1e9f1b()
  iris = load_iris_dataset()
  model = DecisionTreeLearner.new 1, min_size: 25, max_depth: 10
  model.train iris
  root = model.root

  puts "Multi-level Tree", model.to_s

  assert_false root.is_leaf?
  assert_not_nil root.split
  assert_equal 2, root.children.size
end
test_1e9f1b()

Multi-level Tree
{
  "leaf": false,
  "split": "Numeric[petal_width <=> 1]",
  "children": {
    "petal_width < 1": {
      "leaf": false,
      "split": "Numeric[petal_width <=> 0.1]",
      "children": {
        "petal_width < 0.1": {
          "leaf": true,
          "class_distribution": {
          }
        },
        "petal_width >= 0.1": {
          "leaf": false,
          "split": "Numeric[petal_width <=> 0.1]",
          "children": {
            "petal_width < 0.1": {
              "leaf": true,
              "class_distribution": {
              }
            },
            "petal_width >= 0.1": {
              "leaf": false,
              "split": "Numeric[petal_width <=> 0.1]",
              "children": {
                "petal_width < 0.1": {
                  "leaf": true,
                  "class_distribution": {
                  }
                },
                "petal_width >= 0.1": {
                  "leaf": false,
                  "split": "Numeric[petal_wid

## Question 5.2 (5 Points)

Implement ```find_leaf``` which maps an example to an appropriate leaf node. By recursively applying the split tests, find the leaf node. That leaf will be used to calculate the score. 

The ```find_leaf``` method is recursive given a node. It has 2 termination conditions:

1. Node is a leaf
2. Node would have applied a split but there is no child. This happens if a new categorical value was added.


In [39]:
class DecisionTreeLearner
  attr_accessor :positive_class_label
  def predict example
    leaf = find_leaf @root, example
    return leaf.score @positive_class_label
  end

  def evaluate eval_dataset
    examples = eval_dataset["data"]
    examples.map do |example|
      score = predict(example)
      label = example["label"] == @positive_class_label ? 1 : 0
      [score, label]
    end
  end

def find_leaf node, example
    
    if(node.is_leaf?)
      return node
    end
      
    children = node.children
    node_split = node.split

    children.each do |child_node|
      condition = child_node[0]
      child_decision_node = child_node[1]
      test = node_split.test(example)
      if(condition.eql?(test))
        return find_leaf(child_decision_node, example)
      end
    end
    return node
  end
end

:find_leaf

In [40]:
def test_e35c4e()
  iris = load_iris_dataset()
  
  ##Force decision tree to only have 'petal_length' feature
  iris["features"] = %w(petal_length)
  
  learner = DecisionTreeLearner.new 1, min_size: 10, max_depth: 2
  learner.train iris  
  
  x1 = {"features" => {"petal_length" => 1.2}}
  assert_in_delta 0.0, learner.predict(x1), 1e-3, "example 1, class 1"

  x2 = {"features" => {"petal_length" => 3.1}}
  assert_in_delta 0.5, learner.predict(x2), 1e-3, "example 2, class 1"
  
  learner.positive_class_label = 0
  x3 = {"features" => {"petal_length" => 1.2}}
  assert_in_delta 1.0, learner.predict(x3), 1e-3, "example 3, class 0"
end

test_e35c4e()

## Question 5.3 (5 Points)

We introduce a ```Metric``` interface (mixin) which we will use to evaluate the performance of models. Here, we will implement the ```AUCMetric``` which calculates the AUC for a classifier. We would actually like to plot the entire ROC curve, so you will implement the ```roc_curve``` function which will return the false positive, true positive, and AUC values to be used the plot the curve.

The return format for the ```roc_curve``` function is defined as follows:

```ruby
#False positive rates
fp_rates = [0.0, 0.01, 0.02, ..., 1.0]

#True positive rates
tp_rates = [0.0, 0.01, 0.02, ..., 1.0]

#AUC Value
auc = 0.5

return [fp, tp, auc]
```

When plotted, you should see a curve like this:

![example_roc_curve](./roc_curve.png)




In [41]:
module Metric
  def apply scores
  end
end

:apply

In [42]:
def false_positive(scores, t)

  false_positive=0
  false_negative=0
  true_positive=0
  true_negative=0
  total=0
  scores.each do |item|
    next if item==nil
    if item[0]<t and item[1]!=1.0
      true_negative+=1.0
    elsif item[0]>=t and item[1]==1.0
      true_positive+=1.0
    elsif item[0]>=t and item[1]!=1.0
      false_positive+=1.0
    elsif item[0]<t and item[1]==1.0
      false_negative+=1.0
    end 
  end
  
  return (false_positive)/((true_negative+false_positive))
end

:false_positive

In [43]:
def true_positive(scores, t)
  false_positive=0
  false_negative=0
  true_positive=0
  true_negative=0
  total=0
  scores.each do |item|
    next if item==nil
    if item[0]<t and item[1]!=1.0
      true_negative+=1.0
    elsif item[0]>=t and item[1]==1.0
      true_positive+=1.0
    elsif item[0]>=t and item[1]!=1.0
      false_positive+=1.0
    elsif item[0]<t and item[1]==1.0
      false_negative+=1.0
    end 
  end
      
  return (true_positive)/(true_positive+false_negative)
end

:true_positive

In [47]:
class AUCMetric 
  include Metric
  def roc_curve(scores)

    max = scores[0][0]
    min = scores[0][0]

    (scores.size).times do |i|
      if(max < scores[i][0])
        max = scores[i][0]
      end
      if(min > scores[i][0])
        min = scores[i][0]
      end
    end

    cutCount = 100
    auc = 0.0
    prevTPositive = 1.0
    prevFPositive = 1.0
    t = min
    tIncr = (max-min)/cutCount.to_f

    arrayfp=[]
    arraytp=[]

   false_positive(scores, t)
    cutCount.times do |i|
      t += tIncr
      arrayfp << false_positive(scores, t)
      arraytp << true_positive(scores, t)
    end

    cutCount = 100
    auc = 0.0
    prevTPositive = 1.0
    prevFPositive = 1.0
    t = min
    tIncr = (max-min)/cutCount.to_f
    
    cutCount.times do |i|
      t += tIncr
      auc += (-0.5)*(true_positive(scores, t) + prevTPositive)*(false_positive(scores, t) - prevFPositive)
      prevTPositive = true_positive(scores, t)
      prevFPositive = false_positive(scores, t)
    end

    return [arrayfp, arraytp, auc]
  end
  
  def apply scores
    fp, tp, auc = roc_curve scores
    return auc
  end
end

:apply

In [48]:
def test_99112c()
  german_credit = load_german_credit_dataset()
  examples = german_credit["data"]
  learner = DecisionTreeLearner.new 1, min_size: 100, max_depth: 10
  learner.train german_credit

  scores = learner.evaluate german_credit
  metric = AUCMetric.new
  fp, tp, auc = metric.roc_curve scores
  
  puts "AUC = #{auc}"
  
  plot = Daru::DataFrame.new({x: fp, y: tp}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "False Positive Rate"
    plot.y_label "True Positive Rate"
  end
  plot.add(:line, [0,1], [0,1]).color(:gray)
  plot.show()
end

test_99112c()

AUC = 0.7983285714285713


#<Nyaplot::Frame:0x0000000003805360 @properties={:panes=>[#<Nyaplot::Plot:0x000000000392df08 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000003678858 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"ef5a2270-7eb3-4ac6-8d1b-94f86935669e"}, @xrange=[0.0, 0.9533333333333334], @yrange=[0.0, 1.0]>, #<Nyaplot::Diagram:0x0000000003805f90 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:gray}, :data=>"b875202f-ff9c-48b5-bff9-3c136b38343e"}, @xrange=[0, 1], @yrange=[0, 1]>], :options=>{:x_label=>"False Positive Rate", :y_label=>"True Positive Rate", :zoom=>true, :width=>700, :xrange=>[0.0, 1], :yrange=>[0.0, 1.0]}}>], :data=>{"ef5a2270-7eb3-4ac6-8d1b-94f86935669e"=>#<Nyaplot::DataFrame:0x00000000036803c8 @name="ef5a2270-7eb3-4ac6-8d1b-94f86935669e", @rows=[{:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.9533333333333334, :y=>1.0}, {:x=>0.94, :y=>0.9985714285714286}, {:x=>0.94, :y=>0.9985714285714286}, {:x=>0.94, :y=>0.9985714285714286}, {:x=>0.94, :y=>0.9985714285714286}, {:x=>0.9066666666666666, :y=>0.9942857142857143}, {:x=>0.8766666666666667, :y=>0.99}, {:x=>0.8766666666666667, :y=>0.99}, {:x=>0.8766666666666667, :y=>0.99}, {:x=>0.8233333333333334, :y=>0.9814285714285714}, {:x=>0.8066666666666666, :y=>0.9785714285714285}, {:x=>0.8066666666666666, :y=>0.9785714285714285}, {:x=>0.8066666666666666, :y=>0.9785714285714285}, {:x=>0.8066666666666666, :y=>0.9785714285714285}, {:x=>0.8066666666666666, :y=>0.9785714285714285}, {:x=>0.8066666666666666, :y=>0.9785714285714285}, {:x=>0.8066666666666666, :y=>0.9785714285714285}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.6, :y=>0.93}, {:x=>0.5333333333333333, :y=>0.9028571428571428}, {:x=>0.5266666666666666, :y=>0.9}, {:x=>0.5266666666666666, :y=>0.9}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.4533333333333333, :y=>0.8671428571428571}, {:x=>0.39666666666666667, :y=>0.8271428571428572}, {:x=>0.39666666666666667, :y=>0.8271428571428572}, {:x=>0.38, :y=>0.8142857142857143}, {:x=>0.38, :y=>0.8142857142857143}, {:x=>0.37333333333333335, :y=>0.8085714285714286}, {:x=>0.37333333333333335, :y=>0.8085714285714286}, {:x=>0.37333333333333335, :y=>0.8085714285714286}, {:x=>0.37333333333333335, :y=>0.8085714285714286}, {:x=>0.37333333333333335, :y=>0.8085714285714286}, {:x=>0.36, :y=>0.7942857142857143}, {:x=>0.31666666666666665, :y=>0.7442857142857143}, {:x=>0.17666666666666667, :y=>0.58}, {:x=>0.17, :y=>0.5714285714285714}, {:x=>0.17, :y=>0.5714285714285714}, {:x=>0.17, :y=>0.5714285714285714}, {:x=>0.12333333333333334, :y=>0.5014285714285714}, {:x=>0.12333333333333334, :y=>0.5014285714285714}, {:x=>0.11, :y=>0.4785714285714286}, {:x=>0.11, :y=>0.4785714285714286}, {:x=>0.11, :y=>0.4785714285714286}, {:x=>0.1, :y=>0.4

# Question 6

Using the height-limited Decision Tree Learner as a weak learner, we will now implement the ```RandomForestLearner```. The random forest classifier requires a seed, which is provided below.

## Question 6.1 (10 points)
 
The Random Forest algorithm combines Bagging with random features. Given a dataset, we will first identify a random subset of 3 features from the dataset. 


Next, implement Bagging which is to create bootstrap sample. A bootstrap sample is a sample of the same size as the original dataset, but each example is sampled with uniform probability with replacement. Sampling with replacement will result in some of the examples appearing more than once in the final dataset. 

The ```random_forest_dataset``` should return a new dataset where ```data``` and ```features``` have been changed based on the bootstrapping and random features logic.

For example, given a dataset with 4 features, we expect a dataset with 3 features, defined as follows:

```ruby
{
    "labels" => [...],
    "features" => ["x2", "x3", "x4"],
    "data" => [
        {"id"=>92, "features"=>{"x2"=>92, "x4"=>92, "x3"=>92}}, 
        {"id"=>26, "features"=>{"x2"=>26, "x4"=>26, "x3"=>26}}, 
        {"id"=>33, "features"=>{"x2"=>33, "x4"=>33, "x3"=>33}}, 
        ...
    ]
}
```


Notes:

1. Because we are using random numbers it is important that you call ```rng.rand(...)``` the right number of times. Using some other random number generator or even just one extra call will cause the tests not to pass. See Ruby's [Random](https://ruby-doc.org/core-2.5.4/Random.html) class for more details.

In [49]:
SEED = 'eifjcchdivlbcbflbgblfgukbtkhvejvtkevfbtetjnl'.to_i(26)

3073016676488

In [50]:
def random_features_subset dataset, rng
  num_features = 3
  feature_list = dataset["features"].sample(num_features, random: rng)  
end

def random_forest_dataset dataset, rng
  feature_list = random_features_subset dataset, rng
  examples = dataset["data"]
  new_dataset = dataset.clone
  
  bootstrap=[]
  
  dataset["data"].size.times do |i|

    temp=dataset["data"][rng.rand(dataset["data"].size)].clone
    temp["features"]=temp["features"].clone
    temp["features"].each do |key,array|
      temp["features"][key]=temp["features"][key].clone
      if !(feature_list.include?(key))
        temp["features"].delete(key)
      end
    end
    bootstrap << temp
  end
  
  new_dataset["features"]=feature_list
  new_dataset["data"]=bootstrap
  
  return new_dataset
end

:random_forest_dataset

In [51]:
def test_61dee6()
  rng = Random.new(SEED)
  
  examples = Array.new 100 do |id|
    {"id" => id, "features" => {"x1" => id * 1.0, "x2" => id, "x3" => id, "x4" => id}}
  end
  dataset = {"features" => %w(x1 x2 x3 x4), "data" => examples}
  
  rf_dataset = random_forest_dataset dataset, rng
  
  features = rf_dataset["features"].sort
  puts "Sampled features", features
  
  assert_equal 3, features.size
  assert_equal "x2", features[0]  
  assert_equal "x4", features[2]  
end
test_61dee6()

Sampled features
["x2", "x3", "x4"]


In [52]:
def test_e26103()
  rng = Random.new(SEED)
  rng2 = Random.new(SEED)  
  examples = Array.new 100 do |id|
    {"id" => id, "features" => {"x1" => rng2.rand, "x2" => rng2.rand, "x3" => rng2.rand, "x4" => rng2.rand}}
  end
  dataset = {"features" => %w(x1 x2 x3 x4), "data" => examples}
  
  rf_dataset = random_forest_dataset dataset, rng
  
  features = rf_dataset["features"].sort
  puts "Sampled features", features
  
  rf_examples = rf_dataset["data"]
  puts rf_examples[0,5]
  assert_equal 100, rf_examples.size
  assert_equal 92, rf_examples[0]["id"]  
  assert_in_delta 0.7382309556062473, rf_examples[0]["features"]["x2"], 1e-3, "Must copy original example values"
  assert_equal 23, rf_examples[3]["id"]
  assert_in_delta 0.11281946181217362, rf_examples[3]["features"]["x4"], 1e-3, "Must copy original example values"
  
  duplicated_ids = rf_examples.group_by {|e| e["id"]}.select {|k,v| v.size > 1}.to_h.keys.sort
  
  puts "Expect ID #{duplicated_ids.last} is duplicated"
  assert_equal 25, duplicated_ids.size
  assert_equal 91, duplicated_ids.last
end
test_e26103()

Sampled features
["x2", "x3", "x4"]
[{"id"=>92, "features"=>{"x2"=>0.7382309556062473, "x3"=>0.4902051538880293, "x4"=>0.8321339621196061}}, {"id"=>26, "features"=>{"x2"=>0.9058511884689645, "x3"=>0.6480159304053194, "x4"=>0.14862808413212236}}, {"id"=>33, "features"=>{"x2"=>0.8779817106571303, "x3"=>0.5935117988751606, "x4"=>0.052513163798546425}}, {"id"=>23, "features"=>{"x2"=>0.8127461649463698, "x3"=>0.9818758966334915, "x4"=>0.11281946181217362}}, {"id"=>55, "features"=>{"x2"=>0.6965391901897178, "x3"=>0.37992498973029476, "x4"=>0.059710713558340456}}]
Expect ID 91 is duplicated


In [53]:
def test_5682b7()
  rng = Random.new(SEED)
  examples = Array.new 100 do |id|
    {"id" => id, "features" => {"x1" => id * 1.0, "x2" => id, "x3" => id, "x4" => id}}
  end
  dataset = {"features" => %w(x1 x2 x3 x4), "data" => examples}
  
  rf_dataset = random_forest_dataset dataset, rng
  
  features = rf_dataset["features"].sort
  rf_examples = rf_dataset["data"]
  
  assert_equal 100, rf_examples.size
  checked = 0
  rf_examples.each do |e| 
    assert_equal features, e["features"].keys.sort, "Expected only subset of features"
    checked += 1
  end
  assert_equal 100, checked
end
test_5682b7()

## Question 6.2 (2 Points)

Implement the ```train``` method in the ```RandomForestLearner```, which simply calls train on each of the ```@trees``` with the ```random_forest_dataset```.

In [54]:
class RandomForestLearner
  include Learner  
  attr_reader :trees
  
  def initialize positive_class_label, num_trees: 10, min_size: 10, max_depth: 50
    @parameters = {"num_trees" => num_trees, "min_size" => min_size, "max_depth" => max_depth}
    @positive_class_label = positive_class_label
    tree_parameters = @parameters.clone.delete :num_trees
    
    @trees = Array.new(num_trees) do |i| 
    DecisionTreeLearner.new @positive_class_label, min_size: min_size, max_depth: max_depth
    end
  end
  
  def to_s
    JSON.pretty_generate(@trees.collect {|t| t.summarize_node t.root})
  end
  
  def train dataset
    rng = Random.new SEED
   
    @trees.each do |tree|
      train_dataset = random_forest_dataset(dataset, rng)
      tree.train(train_dataset)     
    end
  end
end
  

:train

In [55]:
def test_99f31e()
  iris = load_iris_dataset()
  model = RandomForestLearner.new 2, num_trees: 3, max_depth: 2, min_size: 25
  model.train iris
  trees = model.trees

  puts "Random Forest 3 trees, 2 levels", model.to_s

  assert_equal 3, trees.size
end
test_99f31e()

Random Forest 3 trees, 2 levels
[
  {
    "leaf": false,
    "split": "Numeric[petal_length <=> 3]",
    "children": {
      "petal_length < 3": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        }
      },
      "petal_length >= 3": {
        "leaf": true,
        "class_distribution": {
          "2": 0.5555555555555556,
          "1": 0.4444444444444444
        }
      }
    }
  },
  {
    "leaf": false,
    "split": "Numeric[petal_length <=> 3]",
    "children": {
      "petal_length < 3": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        }
      },
      "petal_length >= 3": {
        "leaf": true,
        "class_distribution": {
          "2": 0.4205607476635514,
          "1": 0.5794392523364486
        }
      }
    }
  },
  {
    "leaf": false,
    "split": "Numeric[petal_length <=> 3]",
    "children": {
      "petal_length < 3": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        

## Question 6.3 (5 Points)

Implement the ```predict``` method in the ```RandomForestLearner```. Define the score for a random forest as follows:

# $P(c \mid t_1, \dots ,t_k) = \frac{1}{k} \sum_{i} P(c \mid t_i)$

where $t_i$ is the $i$th tree and $P(c \mid t_i)$ is the score for the $i$th tree.

In [56]:
class RandomForestLearner
  attr_accessor :positive_class_label
  
  def evaluate eval_dataset
    examples = eval_dataset["data"]
    examples.map do |example|
      score = predict(example)
      label = example["label"] == @positive_class_label ? 1 : 0
      [score, label]
    end
  end
  
  def predict example

    scores = @trees.inject(0.0) {|u, t| u += t.predict(example) }
    scores/(@trees.size).to_f
    
  end
end


:predict

In [57]:
def test_99f31e()
  german_credit = load_german_credit_dataset()
  learner = RandomForestLearner.new 1, num_trees: 11, min_size: 100, max_depth: 10
  learner.train german_credit
  assert_equal 11, learner.trees.size

  scores = learner.evaluate german_credit
  metric = AUCMetric.new
  fp, tp, auc = metric.roc_curve scores
  
  puts "AUC = #{auc}"
  
  plot = Daru::DataFrame.new({x: fp, y: tp}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "False Positive Rate"
    plot.y_label "True Positive Rate"
  end
  plot.add(:line, [0,1], [0,1]).color(:gray)
  plot.show()
end
test_99f31e()

AUC = 0.8311428571428572


#<Nyaplot::Frame:0x0000000003c42158 @properties={:panes=>[#<Nyaplot::Plot:0x0000000003cc9ab8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000003c44048 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"4e2e9b67-d1ad-44c6-a358-6c071a0ccbf9"}, @xrange=[0.0, 0.9966666666666667], @yrange=[0.0014285714285714286, 1.0]>, #<Nyaplot::Diagram:0x0000000003c42b08 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:gray}, :data=>"16978fea-6c13-40d4-b0c1-d4590c240dfa"}, @xrange=[0, 1], @yrange=[0, 1]>], :options=>{:x_label=>"False Positive Rate", :y_label=>"True Positive Rate", :zoom=>true, :width=>700, :xrange=>[0.0, 1], :yrange=>[0, 1.0]}}>], :data=>{"4e2e9b67-d1ad-44c6-a358-6c071a0ccbf9"=>#<Nyaplot::DataFrame:0x0000000003c492c8 @name="4e2e9b67-d1ad-44c6-a358-6c071a0ccbf9", @rows=[{:x=>0.9966666666666667, :y=>1.0}, {:x=>0.9966666666666667, :y=>1.0}, {:x=>0.9966666666666667, :y=>1.0}, {:x=>0.9966666666666667, :y=>1.0}, {:x=>0.9933333333333333, :y=>1.0}, {:x=>0.9933333333333333, :y=>1.0}, {:x=>0.9933333333333333, :y=>1.0}, {:x=>0.9933333333333333, :y=>1.0}, {:x=>0.9933333333333333, :y=>1.0}, {:x=>0.9933333333333333, :y=>1.0}, {:x=>0.9933333333333333, :y=>1.0}, {:x=>0.99, :y=>1.0}, {:x=>0.99, :y=>1.0}, {:x=>0.9866666666666667, :y=>1.0}, {:x=>0.9833333333333333, :y=>1.0}, {:x=>0.98, :y=>1.0}, {:x=>0.9766666666666667, :y=>1.0}, {:x=>0.9766666666666667, :y=>1.0}, {:x=>0.9766666666666667, :y=>1.0}, {:x=>0.9766666666666667, :y=>1.0}, {:x=>0.9733333333333334, :y=>1.0}, {:x=>0.97, :y=>1.0}, {:x=>0.9666666666666667, :y=>1.0}, {:x=>0.9666666666666667, :y=>1.0}, {:x=>0.96, :y=>0.9971428571428571}, {:x=>0.9533333333333334, :y=>0.9971428571428571}, {:x=>0.9466666666666667, :y=>0.9971428571428571}, {:x=>0.9366666666666666, :y=>0.9971428571428571}, {:x=>0.9366666666666666, :y=>0.9957142857142857}, {:x=>0.9166666666666666, :y=>0.9942857142857143}, {:x=>0.9066666666666666, :y=>0.9928571428571429}, {:x=>0.8933333333333333, :y=>0.9914285714285714}, {:x=>0.8833333333333333, :y=>0.99}, {:x=>0.8666666666666667, :y=>0.9885714285714285}, {:x=>0.8566666666666667, :y=>0.9885714285714285}, {:x=>0.8466666666666667, :y=>0.9885714285714285}, {:x=>0.8333333333333334, :y=>0.9857142857142858}, {:x=>0.8066666666666666, :y=>0.9842857142857143}, {:x=>0.7966666666666666, :y=>0.9814285714285714}, {:x=>0.77, :y=>0.9785714285714285}, {:x=>0.7533333333333333, :y=>0.9757142857142858}, {:x=>0.73, :y=>0.97}, {:x=>0.7066666666666667, :y=>0.9614285714285714}, {:x=>0.6733333333333333, :y=>0.9571428571428572}, {:x=>0.6666666666666666, :y=>0.9428571428571428}, {:x=>0.63, :y=>0.9371428571428572}, {:x=>0.5833333333333334, :y=>0.9342857142857143}, {:x=>0.5466666666666666, :y=>0.9242857142857143}, {:x=>0.51, :y=>0.9085714285714286}, {:x=>0.4666666666666667, :y=>0.9}, {:x=>0.44, :y=>0.89}, {:x=>0.41333333333333333, :y=>0.87}, {:x=>0.38666666666666666, :y=>0.8514285714285714}, {:x=>0.36666666666666664, :y=>0.8357142857142857}, {:x=>0.32666666666666666, :y=>0.8214285714285714}, {:x=>0.3, :y=>0.7957142857142857}, {:x=>0.27, :y=>0.7714285714285715}, {:x=>0.24, :y=>0.7428571428571429}, {:x=>0.19333333333333333, :y=>0.71}, {:x=>0.16333333333333333, :y=>0.6757142857142857}, {:x=>0.15, :y=>0.6457142857142857}, {:x=>0.13333333333333333, :y=>0.6085714285714285}, {:x=>0.11666666666666667, :y=>0.5728571428571428}, {:x=>0.10666666666666667, :y=>0.5457142857142857}, {:x=>0.08333333333333333, :y=>0.5157142857142857}, {:x=>0.07333333333333333, :y=>0.4757142857142857}, {:x=>0.06333333333333334, :y=>0.44285714285714284}, {:x=>0.06333333333333334, :y=>0.42}, {:x=>0.06, :y=>0.3914285714285714}, {:x=>0.043333333333333335, :y=>0.37142857142857144}, {:x=>0.04, :y=>0.33714285714285713}, {:x=>0.03, :y=>0.31142857142857144}, {:x=>0.02666666666666667, :y=>0.2842857142857143}, {:x=>0.016666666666666666, :y=>0.25857142857142856}, {:x=>0.013333333333333334, :y=>0.24}, {:x=>0.01, :y=>0.21857142857142858}, {:x=>0.01, :y=>0.1957142857142857}, {:x=>0.0033333333333333335, :y=>0.167142857142

In the tests above, we calculated the AUC for a single decision tree and a random forest, using the same tree depth and minimum leaf size. Is the training AUC better on Random Forest and Decision Tree?

**A**. We cannot compare AUC across different models.

**B**. Both Single Decision Tree and Random Forest perform the same.

**C**. Single Decision Tree has better training AUC that a Random Forest.

**D**. Random Forest has better training AUC that a Single Decision Tree.


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answer is "Z", write the following:_

```ruby
def answer_0b47d3()
    %w(Z)
end
```

In [58]:
def answer_0b47d3()
  %w(D)
end

:answer_0b47d3

In [59]:
assert_not_nil answer_0b47d3()

# Question 7.1 (4 Points)

Paste **your** implementation of ```cross_validate``` from previous assignments. 

In [60]:
def cross_validate dataset, folds, &block
  c_dataset = dataset.clone
  examples = c_dataset["data"]
  examples_size = examples.size().to_f
  fold_size = (examples_size / folds).floor
  folds_array = examples.each_slice(fold_size).to_a
 
  folds_array.each_with_index do |value, index|
    test_set = dataset.clone
    test_set["data"] = value
    train_set = dataset.clone
       
    aux_train_data = folds_array.clone
    aux_train_data.delete_at(index)
    train_set["data"] = aux_train_data.flatten(1)
    yield train_set, test_set, index
  end
   
end

:cross_validate

## Question 7.2 (5 points)

Using cross validation and the generic ```Learner```s and ```Metric```s you implemented, we can build a generic tool for parameter selection. The next few methods are implemented for you so that you can interpret the results. 


In [61]:
def cross_validation_model_performance dataset, folds, learners, metric    
  learners.map do |learner|
    tr_metrics = []
    te_metrics = []
    puts "#{folds}-fold CV: #{learner.class.name}, parameters: #{learner.parameters}"
    cross_validate dataset, folds do |train_dataset, test_dataset|
      learner.train train_dataset
      train_scores = learner.evaluate train_dataset
      test_scores = learner.evaluate test_dataset      
      tr_metrics << metric.apply(train_scores)
      te_metrics << metric.apply(test_scores)
    end
    {
      "learner" => learner.class.name, "parameters" => learner.parameters, "folds" => folds,
      "mean_train_metric" => mean(tr_metrics), "stdev_train_metric" => stdev(tr_metrics),
      "mean_test_metric" => mean(te_metrics), "stdev_test_metric" => stdev(te_metrics),
    }
  end
end


:cross_validation_model_performance

In [62]:
def best_performance_by_learner stats  
  stats.group_by {|s| s["learner"]}.map do |g_s|
    learner, learner_stats = g_s
    best_parameters = learner_stats.max_by {|l| l["mean_test_metric"]}    
    [learner, best_parameters]
  end.to_h
end

def parameter_search learners, dataset
  metric = AUCMetric.new  
  stats = cross_validation_model_performance dataset, 5, learners, metric
  best_by_learner = best_performance_by_learner stats  
  puts JSON.pretty_generate(best_by_learner)

  assert_equal learners.size, stats.size
  assert_true(stats.all? {|s| a = s["mean_train_metric"]; a >= 0.0 and a <= 1.0}, "0 <= Train AUC <= 1")
  assert_true(stats.all? {|s| a = s["mean_test_metric"]; a >= 0.0 and a <= 1.0}, "0 <= Train AUC <= 1")
  
  df = Daru::DataFrame.new(stats) 
end

:parameter_search

We will select the best-performing model on the iris using a single decision tree with various ```min_size``` parameter values. 

In [63]:
def test_6774f4()
  iris = load_iris_dataset()
  limit = iris["data"].size
  min_sizes = [1, 2, 5, 10, 15, 20, 25, 50, 100, 200, 250, 500]
  max_depth = 50  
  min_sizes.select! {|s| s < limit}

  learners = min_sizes.map do |s|
    DecisionTreeLearner.new(1, min_size: s, max_depth: max_depth)
  end

  parameter_search learners, iris
end
test_6774f4()

5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>1, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>2, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>5, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>10, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>15, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>20, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>25, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>50, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>100, "max_depth"=>50}
{
  "DecisionTreeLearner": {
    "learner": "DecisionTreeLearner",
    "parameters": {
      "min_size": 5,
      "max_depth": 50
    },
    "folds": 5,
    "mean_train_metric": 0.9989665620143364,
    "stdev_train_metric": 0.0005178575170429548,
    "mean_test_metric": 0.9669308980493192,
    "stde

,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,DecisionTreeLearner,"{""min_size""=>1, ""max_depth""=>50}",5,1.0,0.0,0.9345445344129555,0.0733671773349088
1,DecisionTreeLearner,"{""min_size""=>2, ""max_depth""=>50}",5,1.0,0.0,0.9345445344129555,0.0733671773349088
2,DecisionTreeLearner,"{""min_size""=>5, ""max_depth""=>50}",5,0.9989665620143364,0.0005178575170429548,0.9669308980493192,0.027307119579043134
3,DecisionTreeLearner,"{""min_size""=>10, ""max_depth""=>50}",5,0.9982473228658805,0.0007191887672458989,0.966430898049319,0.028272644062163847
4,DecisionTreeLearner,"{""min_size""=>15, ""max_depth""=>50}",5,0.9982473228658805,0.0007191887672458989,0.966430898049319,0.028272644062163847
5,DecisionTreeLearner,"{""min_size""=>20, ""max_depth""=>50}",5,0.9982473228658805,0.0007191887672458989,0.966430898049319,0.028272644062163847
6,DecisionTreeLearner,"{""min_size""=>25, ""max_depth""=>50}",5,0.9982473228658805,0.0007191887672458989,0.966430898049319,0.028272644062163847
7,DecisionTreeLearner,"{""min_size""=>50, ""max_depth""=>50}",5,0.9625302528281111,0.019007707795564037,0.9396210461365261,0.0625609289874913
8,DecisionTreeLearner,"{""min_size""=>100, ""max_depth""=>50}",5,0.599259745534846,0.13907511990779564,0.600458767238953,0.13181465182371033


Select the best-performing model using RandomForest on the iris dataset with various ```min_size``` parameters.

In [64]:
def test_f9aacd()
  iris = load_iris_dataset()
  limit = iris["data"].size
  min_sizes = [1, 2, 5, 10, 15, 20, 25, 50, 100, 200, 250, 500]
  max_depth = 50  
  num_trees = 11
  min_sizes.select! {|s| s < limit}

  learners = min_sizes.map do |s|
    RandomForestLearner.new(1, num_trees: num_trees, min_size: s, max_depth: max_depth)
  end

  parameter_search learners, iris
end
test_f9aacd()

5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>1, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>2, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>5, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>10, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>15, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>20, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>25, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>50, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>100, "max_depth"=>50}
{
  "RandomForestLearner": {
    "learner": "RandomForestLearner",
    "parameters": {
      "num_trees": 11,
      "min_size": 5,
      "max_depth

,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>1, ""max_depth""=>50}",5,1.0,9.614813431917819e-17,0.9902930622009569,0.013082264154270898
1,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>2, ""max_depth""=>50}",5,1.0,7.850462293418876e-17,0.9919976076555024,0.00968866293954433
2,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>5, ""max_depth""=>50}",5,0.9998127745115697,0.0002754436926681575,0.9959066985645932,0.0042249474383316
3,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>10, ""max_depth""=>50}",5,0.9987143386987765,0.0005789812307288406,0.9950536816125052,0.004791949888287957
4,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>15, ""max_depth""=>50}",5,0.9981451952581469,0.001014202784658836,0.9950536816125052,0.004791949888287957
5,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>20, ""max_depth""=>50}",5,0.9981451952581469,0.001014202784658836,0.9950536816125052,0.004791949888287957
6,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>25, ""max_depth""=>50}",5,0.9981451952581469,0.001014202784658836,0.9950536816125052,0.004791949888287957
7,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>50, ""max_depth""=>50}",5,0.9953056230166757,0.0020947114451937312,0.989371863430687,0.013957991017633445
8,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>100, ""max_depth""=>50}",5,0.862281699359929,0.1165309677630377,0.8439845850743684,0.14280195935156728


We will select the best-performing model on the german credit dataset using a single decision tree with various ```min_size``` parameter values. 

In [65]:
def test_0678f4()
  german_credit = load_german_credit_dataset()
  limit = german_credit["data"].size
  min_sizes = [1, 2, 5, 10, 15, 20, 25, 50, 100, 200, 250, 500]
  max_depth = 50  
  min_sizes.select! {|s| s < limit}

  learners = min_sizes.map do |s|
    DecisionTreeLearner.new(1, min_size: s, max_depth: max_depth)
  end
  parameter_search learners, german_credit
end
test_0678f4()

5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>1, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>2, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>5, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>10, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>15, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>20, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>25, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>50, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>100, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>200, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>250, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>500, "max_depth"=>50}
{
  "DecisionTreeLearner": {
    "learner": "DecisionTreeLearne

,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,DecisionTreeLearner,"{""min_size""=>1, ""max_depth""=>50}",5,1.0,0.0,0.6516006091556481,0.01918964319590248
1,DecisionTreeLearner,"{""min_size""=>2, ""max_depth""=>50}",5,1.0,0.0,0.645453895387433,0.01711775798586906
2,DecisionTreeLearner,"{""min_size""=>5, ""max_depth""=>50}",5,0.9917067985565085,0.0017178055442335148,0.663407636652569,0.026003465696895333
3,DecisionTreeLearner,"{""min_size""=>10, ""max_depth""=>50}",5,0.9578304448517698,0.0059235984382986235,0.6802193422234561,0.03286941967312497
4,DecisionTreeLearner,"{""min_size""=>15, ""max_depth""=>50}",5,0.9247815762488925,0.009202376822055365,0.6903447461045598,0.027812012365180622
5,DecisionTreeLearner,"{""min_size""=>20, ""max_depth""=>50}",5,0.9031451380462301,0.006733180130797864,0.6946629160908199,0.010637289149384473
6,DecisionTreeLearner,"{""min_size""=>25, ""max_depth""=>50}",5,0.8901014298504565,0.008473446168602494,0.7035040472769863,0.01716436804579295
7,DecisionTreeLearner,"{""min_size""=>50, ""max_depth""=>50}",5,0.8212790476231632,0.013079992195128268,0.7035813184411198,0.023590324118570035
8,DecisionTreeLearner,"{""min_size""=>100, ""max_depth""=>50}",5,0.7954524902729203,0.014909288040687911,0.7112051532135919,0.01604189606605205
9,DecisionTreeLearner,"{""min_size""=>200, ""max_depth""=>50}",5,0.7769207959173894,0.012726377745378778,0.7047543476518368,0.016109889999225177


In [66]:
def test_f3ac87()
  german_credit = load_german_credit_dataset()
  limit = german_credit["data"].size
  min_sizes = [1, 2, 5, 10, 15, 20, 25, 50, 100, 200, 250, 500]
  max_depth = 50  
  num_trees = 11
  min_sizes.select! {|s| s < limit}

  learners = min_sizes.map do |s|
      RandomForestLearner.new(1, num_trees: num_trees, min_size: s, max_depth: max_depth)
  end
  parameter_search learners, german_credit
end
test_f3ac87()

5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>1, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>2, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>5, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>10, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>15, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>20, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>25, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>50, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>100, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>200, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_t

,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>1, ""max_depth""=>50}",5,0.9433715915320612,0.006533180048217666,0.6976021250313444,0.024394464967431646
1,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>2, ""max_depth""=>50}",5,0.9438138107947645,0.006918617113953533,0.6971035902592824,0.02071717414106228
2,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>5, ""max_depth""=>50}",5,0.9344985357821514,0.0070194704241788265,0.6980830205450256,0.02437193865912593
3,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>10, ""max_depth""=>50}",5,0.9183717945414595,0.008618569518239206,0.7107811422708211,0.01833642476446939
4,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>15, ""max_depth""=>50}",5,0.908521553196698,0.008820135563858142,0.7139822552227277,0.027820610290835726
5,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>20, ""max_depth""=>50}",5,0.8967379934105558,0.010389539542732244,0.7168904816774109,0.023072017113250853
6,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>25, ""max_depth""=>50}",5,0.8892297840181689,0.009162207406350698,0.7268518942266013,0.027861110298737675
7,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>50, ""max_depth""=>50}",5,0.8531862682882319,0.0037218511366347413,0.7425323643236931,0.009877903173570856
8,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>100, ""max_depth""=>50}",5,0.822898216058182,0.0037116306389919063,0.7508894707978606,0.011038725643712746
9,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>200, ""max_depth""=>50}",5,0.8006474191295838,0.006417435439972225,0.7559961281325124,0.007551962111232829


Next, let's see the effect of increasing the number of trees for the iris dataset

In [67]:
def test_10cb92()
  german_credit = load_iris_dataset()
  min_size = 50
  max_depth = 3
  num_trees = [1,2,3,5,7,9,11, 21, 51, 101, 151, 201]
  
  learners = num_trees.map do |s|
    RandomForestLearner.new(1, num_trees: s, min_size: min_size, max_depth: max_depth)
  end
  parameter_search learners, german_credit
end
test_10cb92()

5-fold CV: RandomForestLearner, parameters: {"num_trees"=>1, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>2, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>3, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>5, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>7, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>9, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>21, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>51, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>101, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>151, "m

,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,RandomForestLearner,"{""num_trees""=>1, ""min_size""=>50, ""max_depth""=>3}",5,0.9508899977474276,0.025466579382242265,0.8807260711424798,0.07227791155402262
1,RandomForestLearner,"{""num_trees""=>2, ""min_size""=>50, ""max_depth""=>3}",5,0.9795671549567132,0.0035564916897411203,0.9399988741908247,0.045104728954140846
2,RandomForestLearner,"{""num_trees""=>3, ""min_size""=>50, ""max_depth""=>3}",5,0.9796730796666372,0.012872718196133656,0.9453948072051788,0.05164726111193679
3,RandomForestLearner,"{""num_trees""=>5, ""min_size""=>50, ""max_depth""=>3}",5,0.9910529192214266,0.008001159438338649,0.9834635302777718,0.017399811212524697
4,RandomForestLearner,"{""num_trees""=>7, ""min_size""=>50, ""max_depth""=>3}",5,0.9911851811006764,0.005557217345076221,0.9875619086795556,0.014285071965869834
5,RandomForestLearner,"{""num_trees""=>9, ""min_size""=>50, ""max_depth""=>3}",5,0.9945619304613615,0.002203274041863197,0.9884668860551212,0.013976747722835236
6,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>50, ""max_depth""=>3}",5,0.9946862916968339,0.002158615707251241,0.9884668860551212,0.013976747722835236
7,RandomForestLearner,"{""num_trees""=>21, ""min_size""=>50, ""max_depth""=>3}",5,0.9958251362492913,0.001601850000398694,0.9916445907034142,0.009345627427854108
8,RandomForestLearner,"{""num_trees""=>51, ""min_size""=>50, ""max_depth""=>3}",5,0.9967801980342141,0.0017083426278939906,0.9916445907034142,0.009345627427854114
9,RandomForestLearner,"{""num_trees""=>101, ""min_size""=>50, ""max_depth""=>3}",5,0.9967164812725807,0.0016597217781951956,0.9950536816125052,0.004791949888287986


## Question 7.3 (2 Points)

In ```test_6774f4``` above, we test different min sizes for the iris dataset. Considering the results for that test, which of the following are true?

**A**. The best performance on the testing set was ```min_size = 150```.

**B**. The best performance on the testing set was ```min_size = 1```.

**C**. The best performance on the training set was ```min_size = 1```.

**D**. The best performance on the testing set was neither 150 nor 1.


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answers are "E" and "F", write the following:_

```ruby
def answer_fa007f()
    %w(E F)
end
```

In [68]:
def answer_fa007f()
  %w(C D)
end

:answer_fa007f

In [69]:
assert_not_nil answer_fa007f()

## Question 7.4 (2 Points)

In ```test_0678f4``` and ```test_f3ac87``` above, we test different min sizes for the german credit dataset for a single Decision Tree and a random forest. Considering the results for these tests, which of the following are true?

**A**. RandomForest has better average test AUC than a single Decision Tree.

**B**. RandomForest has better average train AUC than a single Decision Tree.

**C**. RandomForest has lower standard deviation test AUC than a single Decision Tree.

**D**. RandomForest has higher standard deviation test AUC than a single Decision Tree.


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answers are "E" and "F", write the following:_

```ruby
def answer_545512()
    %w(E F)
end
```

In [70]:
def answer_545512()
  %w(A C)
end

:answer_545512

In [71]:
assert_not_nil answer_545512()

## Question 7.5 (2 Points)

In ```test_10cb92``` above, we tested different number of trees for a random forest on the iris dataset. Considering the results for these tests, which of the following are true?

**A**. Increasing the number of trees makes no difference for the iris dataset in this test.

**B**. RandomForest testing AUC usually does not become worse with increasing number of trees.

**C**. The average testing AUC is monotonically decreasing as the number of trees increases.

**D**. Increasing the number of trees led to overfitting for the iris dataset.


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answers is "Z", write the following:_

```ruby
def answer_ce174b()
    %w(Z)
end
```

In [72]:
def answer_ce174b()
  %w(B)
end

:answer_ce174b

In [73]:
assert_not_nil answer_ce174b()

RuntimeError: no message received

Interrupt: 